# UCI-HAR CNN

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from tensorflow.keras.utils import to_categorical

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


## Load and format UCI-HAR dataset

In [2]:
x_train = np.loadtxt('x_train.csv', delimiter=',').reshape((-1, 128, 9))
y_train = np.loadtxt('y_train.csv', delimiter=',')
x_test = np.loadtxt('x_test.csv', delimiter=',').reshape((-1, 128, 9))
y_test = np.loadtxt('y_test.csv', delimiter=',')
np.savetxt('x_test_250.csv', x_test.reshape((x_test.shape[0], -1))[0:250], delimiter=',', fmt='%s')
np.savetxt('y_test_250.csv', y_test[0:250], delimiter=',', fmt='%s')
x_test_250 = np.loadtxt('x_test_250.csv', delimiter=',').reshape((-1, 128, 9))
y_test_250 = np.loadtxt('y_test_250.csv', delimiter=',')

## Build model

In [3]:
model = Sequential()
model.add(Input(shape=(128, 9)))
model.add(Conv1D(filters=2, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on

# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(lr=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 127, 2)            38        
_________________________________________________________________
flatten (Flatten)            (None, 254)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1530      
Total params: 1,568
Trainable params: 1,568
Non-trainable params: 0
_________________________________________________________________


C:\Users\Laurent Rodriguez\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


## Train model

In [4]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test_250, y_test_250))

Epoch 1/3


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv1d/conv1d (defined at \LAUREN~1\AppData\Local\Temp/ipykernel_14908/1497882645.py:1) ]] [Op:__inference_train_function_621]

Function call stack:
train_function


## Evaluate model on small dataset

In [121]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

8/8 - 0s - loss: 1.1556 - categorical_accuracy: 0.5520
tf.Tensor(
[[223 123  43  25  52   0]
 [ 64  92  15  20 255   0]
 [156  48 106  80   6   0]
 [ 73  29  19 192 155   0]
 [  4  11   4  15 470   0]
 [  0   0   0   0  27 486]], shape=(6, 6), dtype=int32)


## Evaluate model on complete dataset

In [75]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

88/88 - 0s - loss: 0.4677 - categorical_accuracy: 0.8543
tf.Tensor(
[[410  48   3   0   5   0]
 [  4 436   3   0   3   0]
 [  6 138 252   0   0   0]
 [  2   7   0 362  96   1]
 [  2   2   0  86 413   1]
 [  0   0   0   0   0 513]], shape=(6, 6), dtype=int32)


## Save trained model

In [7]:
model.save('uci-har_si4.h5')